In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Crohns_Disease"
cohort = "GSE207022"

# Input paths
in_trait_dir = "../../input/GEO/Crohns_Disease"
in_cohort_dir = "../../input/GEO/Crohns_Disease/GSE207022"

# Output paths
out_data_file = "../../output/preprocess/Crohns_Disease/GSE207022.csv"
out_gene_data_file = "../../output/preprocess/Crohns_Disease/gene_data/GSE207022.csv"
out_clinical_data_file = "../../output/preprocess/Crohns_Disease/clinical_data/GSE207022.csv"
json_path = "../../output/preprocess/Crohns_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Efficacy and safety of ustekinumab treatment in patients with Crohn's disease"
!Series_summary	"UNITI-2 was a phase 3 clinical trial (ClinicalTrials.gov Identifier: NCT01369342) comparing the effects (both positive and negative) of an initial treatment with ustekinumab to a placebo over 8 weeks in patients with moderately to severely active Crohn's disease."
!Series_overall_design	"A gene expression profiling study was conducted in which rectum biopsy samples were collected for RNA extraction and hybridization to microarrays from patients (n=125) with moderate-to-severe Crohn's disease and from non-IBD subjects (n=23)."
Sample Characteristics Dictionary:
{0: ['tissue: rectum'], 1: ['donor id: CNTO1275CRD3002-20554', 'donor id: CNTO1275CRD3002-20667', 'donor id: CNTO1275CRD3002-20449', 'donor id: CNTO1275CRD3002-20927', 'donor id: CNTO1275CRD3002-20270', 'donor id: CNTO1275CRD3002-20072', 'donor id: CNTO1275CRD3002-20109', 'donor id: CNTO1275CRD300

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Determine gene expression data availability
is_gene_available = True  # This appears to be gene expression data from microarrays

# 2. Determine clinical data availability and create conversion functions

# 2.1 Identify rows containing relevant clinical data
trait_row = 3  # 'diagnosis: Crohn's disease', 'diagnosis: healthy control'
age_row = None  # Age is not available in the sample characteristics
gender_row = None  # Gender is not available in the sample characteristics

# 2.2 Create conversion functions for clinical variables

def convert_trait(val: str) -> int:
    """Convert Crohn's disease status to binary."""
    if val is None:
        return None
    if ':' in val:
        val = val.split(':', 1)[1].strip().lower()
    else:
        val = val.strip().lower()
    
    if "crohn" in val:
        return 1  # Has Crohn's disease
    elif "healthy" in val or "control" in val:
        return 0  # Healthy control
    return None

def convert_age(val: str) -> Optional[float]:
    """Convert age to continuous value."""
    # Not used since age data is not available
    return None

def convert_gender(val: str) -> Optional[int]:
    """Convert gender to binary (0 for female, 1 for male)."""
    # Not used since gender data is not available
    return None

# 3. Save initial metadata about dataset usability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if trait data is available
if trait_row is not None:
    # Create the clinical data DataFrame from the sample characteristics dictionary
    # Based on the previous output, we have samples in columns and characteristics in rows
    sample_characteristics = {
        0: ['tissue: rectum'], 
        1: ['donor id: CNTO1275CRD3002-20554', 'donor id: CNTO1275CRD3002-20667', 'donor id: CNTO1275CRD3002-20449', 
            'donor id: CNTO1275CRD3002-20927', 'donor id: CNTO1275CRD3002-20270', 'donor id: CNTO1275CRD3002-20072', 
            'donor id: CNTO1275CRD3002-20109', 'donor id: CNTO1275CRD3002-20346', 'donor id: HC-1', 'donor id: HC-2', 
            'donor id: HC-3', 'donor id: HC-4', 'donor id: HC-5', 'donor id: HC-6', 'donor id: HC-7', 'donor id: HC-8', 
            'donor id: HC-9', 'donor id: HC-10', 'donor id: HC-11', 'donor id: HC-12', 'donor id: HC-13', 'donor id: HC-14', 
            'donor id: HC-15', 'donor id: HC-16', 'donor id: HC-17', 'donor id: HC-18', 'donor id: HC-19', 'donor id: HC-20', 
            'donor id: HC-21', 'donor id: HC-22'], 
        2: ['visit: I-WK0'], 
        3: ["diagnosis: Crohn's disease", 'diagnosis: healthy control'], 
        4: ['treatment: Ustekinumab 130 mg IV', 'treatment: Ustekinumab 6 mg/kg (520 mg)', 'treatment: Placebo IV', 
            'treatment: Ustekinumab 6 mg/kg (390 mg)', 'treatment: NA', 'treatment: Ustekinumab 6 mg/kg (260 mg)'], 
        5: ['inflamed area at week 0: Ileum and colon', 'inflamed area at week 0: Colon only', 'inflamed area at week 0: NA'], 
        6: ['mucosal healing at week 8: N', 'mucosal healing at week 8: NA', 'mucosal healing at week 8: Y']
    }
    
    # Let's convert this dictionary to a DataFrame that can be used with geo_select_clinical_features
    # The function expects rows as sample characteristics and columns as samples
    
    # Create a list of sample IDs from the donor id row (row 1)
    sample_ids = []
    for donor_id in sample_characteristics[1]:
        if ':' in donor_id:
            sample_id = donor_id.split(':', 1)[1].strip()
            sample_ids.append(sample_id)
    
    # Initialize a DataFrame with characteristics as rows and samples as columns
    clinical_data = pd.DataFrame(index=range(len(sample_characteristics)), columns=sample_ids)
    
    # Fill the DataFrame with characteristic values
    for row_idx, values in sample_characteristics.items():
        # Handle the case where one row has multiple values (different for different samples)
        if len(values) == 1:
            # Same value for all samples
            for col in clinical_data.columns:
                clinical_data.iloc[row_idx, clinical_data.columns.get_loc(col)] = values[0]
        else:
            # Different values for different samples or less values than samples
            # For diagnosis (row 3), we need to infer values from donor IDs
            if row_idx == 3:  # Diagnosis row
                for col_idx, col in enumerate(clinical_data.columns):
                    if 'HC-' in col:  # Healthy control
                        clinical_data.iloc[row_idx, col_idx] = 'diagnosis: healthy control'
                    else:  # Crohn's disease
                        clinical_data.iloc[row_idx, col_idx] = "diagnosis: Crohn's disease"
            else:
                # For other rows, we'll set to None as we don't have enough information to map
                for col_idx, col in enumerate(clinical_data.columns):
                    clinical_data.iloc[row_idx, col_idx] = None
    
    # Extract clinical features
    selected_clinical = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'CNTO1275CRD3002-20554': [1.0], 'CNTO1275CRD3002-20667': [1.0], 'CNTO1275CRD3002-20449': [1.0], 'CNTO1275CRD3002-20927': [1.0], 'CNTO1275CRD3002-20270': [1.0], 'CNTO1275CRD3002-20072': [1.0], 'CNTO1275CRD3002-20109': [1.0], 'CNTO1275CRD3002-20346': [1.0], 'HC-1': [0.0], 'HC-2': [0.0], 'HC-3': [0.0], 'HC-4': [0.0], 'HC-5': [0.0], 'HC-6': [0.0], 'HC-7': [0.0], 'HC-8': [0.0], 'HC-9': [0.0], 'HC-10': [0.0], 'HC-11': [0.0], 'HC-12': [0.0], 'HC-13': [0.0], 'HC-14': [0.0], 'HC-15': [0.0], 'HC-16': [0.0], 'HC-17': [0.0], 'HC-18': [0.0], 'HC-19': [0.0], 'HC-20': [0.0], 'HC-21': [0.0], 'HC-22': [0.0]}
Clinical data saved to ../../output/preprocess/Crohns_Disease/clinical_data/GSE207022.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at',
       '1294_PM_at', '1316_PM_at', '1320_PM_at', '1405_PM_i_at', '1431_PM_at',
       '1438_PM_at', '1487_PM_at', '1494_PM_f_at', '1552256_PM_a_at',
       '1552257_PM_a_at', '1552258_PM_at', '1552261_PM_at', '1552263_PM_at',
       '1552264_PM_a_at', '1552266_PM_at'],
      dtype='object', name='ID')

Gene data dimensions: 54715 genes × 148 samples


### Step 4: Gene Identifier Review

In [5]:
# Looking at the identifiers: '1007_PM_s_at', '1053_PM_at', etc.
# These are Affymetrix probe IDs from a microarray platform, not human gene symbols
# They need to be mapped to standard gene symbols for analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observe gene identifiers in gene expression data and annotation data
# From previous steps we see:
# - Gene expression data has identifiers like '1007_PM_s_at' in the index
# - Gene annotation data has 'ID' column with the same format identifiers
# - Gene symbols are stored in the 'Gene Symbol' column in the annotation data

# 2. Extract gene mapping from annotation dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# 3. Convert probe-level measurements to gene-level expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print out information about the mapping process
print(f"Original probe data shape: {gene_data.shape[0]} probes × {gene_data.shape[1]} samples")
print(f"Mapped gene data shape: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
print("\nFirst 10 gene symbols after mapping:")
print(gene_data.index[:10])

# Verify first few rows of the mapped gene data
print("\nPreview of gene expression data after mapping:")
print(preview_df(gene_data))


Original probe data shape: 18989 probes × 148 samples
Mapped gene data shape: 18989 genes × 148 samples

First 10 gene symbols after mapping:
Index(['A1BG', 'A1CF', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT',
       'AAA1', 'AAAS'],
      dtype='object', name='Gene')

Preview of gene expression data after mapping:
{'GSM6268367': [2.71, 6.53, 12.6, 6.15, 14.239999999999998], 'GSM6268368': [2.99, 8.16, 12.69, 6.15, 13.29], 'GSM6268369': [3.27, 6.86, 12.58, 7.17, 14.47], 'GSM6268370': [2.81, 8.51, 13.43, 6.57, 12.69], 'GSM6268371': [2.65, 8.66, 14.03, 6.19, 13.02], 'GSM6268372': [2.63, 7.27, 12.760000000000002, 6.52, 13.45], 'GSM6268373': [2.73, 8.3, 13.530000000000001, 6.13, 13.530000000000001], 'GSM6268374': [2.74, 8.21, 13.59, 6.619999999999999, 13.52], 'GSM6268375': [2.91, 7.81, 13.21, 6.03, 13.809999999999999], 'GSM6268376': [2.74, 8.43, 14.37, 6.51, 13.33], 'GSM6268377': [2.65, 8.33, 13.76, 6.779999999999999, 12.84], 'GSM6268378': [2.6, 8.3, 15.07, 6.8, 12.850000000000001],

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data_normalized.shape[0]} genes × {gene_data_normalized.shape[1]} samples")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 2. Read the clinical features from the previously saved file
print("Loading clinical features...")
clinical_features = pd.read_csv(out_clinical_data_file, index_col=0)
print(f"Clinical features shape: {clinical_features.shape}")
print("Clinical features preview:")
print(preview_df(clinical_features))

# First, let's look at the column names of both datasets to ensure proper linking
print("\nGene data columns (first 5):", gene_data_normalized.columns[:5].tolist())
print("Clinical data rows:", clinical_features.index.tolist())

# Since we've detected issues with data linking, let's manually inspect the data formats
# and make necessary adjustments for proper alignment
if clinical_features.shape[0] == 0:
    print("Error: Clinical features dataframe is empty. Cannot proceed with linking.")
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,
        df=pd.DataFrame(),
        note="Clinical features dataframe is empty, cannot link with gene data."
    )
else:
    # Re-extract the clinical data directly from the matrix file
    print("Re-extracting clinical data from the original source...")
    # Get background information and clinical data again
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, original_clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # Extract clinical features properly
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=original_clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    print("Re-extracted clinical features preview:")
    print(preview_df(selected_clinical_df))
    print(f"Re-extracted clinical data shape: {selected_clinical_df.shape}")
    
    # Save the properly extracted clinical features
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Updated clinical features saved to {out_clinical_data_file}")
    
    # 2. Link clinical and genetic data using the re-extracted clinical data
    print("Linking clinical and genetic data...")
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data_normalized)
    print(f"Linked data shape: {linked_data.shape}")
    
    # Check if the linked data has adequate data
    if linked_data.shape[0] == 0 or linked_data.shape[1] <= 4:  # 4 is an arbitrary small number
        print("Error: Linked data has insufficient samples or features. Dataset cannot be processed further.")
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=True,
            is_biased=True,
            df=linked_data,
            note="Failed to properly link gene expression data with clinical features."
        )
    else:
        # 3. Handle missing values systematically
        print("Handling missing values...")
        linked_data_clean = handle_missing_values(linked_data, trait_col=trait)
        print(f"Data shape after handling missing values: {linked_data_clean.shape}")
        
        # Check if there are still samples after missing value handling
        if linked_data_clean.shape[0] == 0:
            print("Error: No samples remain after handling missing values.")
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=True,
                df=pd.DataFrame(),
                note="All samples were removed during missing value handling."
            )
        else:
            # 4. Check if the dataset is biased
            print("\nChecking for bias in feature variables:")
            is_biased, linked_data_final = judge_and_remove_biased_features(linked_data_clean, trait)
            
            # 5. Conduct final quality validation
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=is_biased,
                df=linked_data_final,
                note="Dataset contains gene expression data for Crohn's Disease subtypes (penetrating vs stricturing)."
            )
            
            # 6. Save linked data if usable
            if is_usable:
                os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                linked_data_final.to_csv(out_data_file)
                print(f"Linked data saved to {out_data_file}")
                print(f"Final dataset shape: {linked_data_final.shape}")
            else:
                print("Dataset deemed not usable for trait association studies, linked data not saved.")

Normalizing gene symbols...
Gene data shape after normalization: 18622 genes × 148 samples


Normalized gene expression data saved to ../../output/preprocess/Crohns_Disease/gene_data/GSE207022.csv
Loading clinical features...
Clinical features shape: (1, 30)
Clinical features preview:
{'CNTO1275CRD3002-20554': [1.0], 'CNTO1275CRD3002-20667': [1.0], 'CNTO1275CRD3002-20449': [1.0], 'CNTO1275CRD3002-20927': [1.0], 'CNTO1275CRD3002-20270': [1.0], 'CNTO1275CRD3002-20072': [1.0], 'CNTO1275CRD3002-20109': [1.0], 'CNTO1275CRD3002-20346': [1.0], 'HC-1': [0.0], 'HC-2': [0.0], 'HC-3': [0.0], 'HC-4': [0.0], 'HC-5': [0.0], 'HC-6': [0.0], 'HC-7': [0.0], 'HC-8': [0.0], 'HC-9': [0.0], 'HC-10': [0.0], 'HC-11': [0.0], 'HC-12': [0.0], 'HC-13': [0.0], 'HC-14': [0.0], 'HC-15': [0.0], 'HC-16': [0.0], 'HC-17': [0.0], 'HC-18': [0.0], 'HC-19': [0.0], 'HC-20': [0.0], 'HC-21': [0.0], 'HC-22': [0.0]}

Gene data columns (first 5): ['GSM6268367', 'GSM6268368', 'GSM6268369', 'GSM6268370', 'GSM6268371']
Clinical data rows: ['Crohns_Disease']
Re-extracting clinical data from the original source...
Re-extracte

Data shape after handling missing values: (148, 18623)

Checking for bias in feature variables:
For the feature 'Crohns_Disease', the least common label is '0.0' with 23 occurrences. This represents 15.54% of the dataset.
The distribution of the feature 'Crohns_Disease' in this dataset is fine.



Linked data saved to ../../output/preprocess/Crohns_Disease/GSE207022.csv
Final dataset shape: (148, 18623)
